In [3]:
%cd ./pytorch_bert_crf_ner

/home/work/nlp/keyword/pytorch_bert_crf_ner


In [ ]:
!pip install -r requirements.txt 

In [2]:
!pip install gluonnlp
!pip install mxnet
!pip install transformers -U

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
!pip install transformers==3.5.1
!pip install sentencepiece==0.1.95
!pip install pytorch-crf==0.7.0
!pip install h5py==2.10.0

In [ ]:
!pip install opencv-python==4.5.5.64

In [ ]:
!python inference.py --data_dir "/home/work/nlp/keyword/model" --model_dir "/home/work/nlp/keyword/model"

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import json
import pickle
import argparse
import torch
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer
from model.net import KobertSequenceFeatureExtractor, KobertCRF, KobertBiLSTMCRF, KobertBiGRUCRF
from gluonnlp.data import SentencepieceTokenizer
from data_utils.utils import Config
from data_utils.vocab_tokenizer import Tokenizer
from data_utils.pad_sequence import keras_pad_fn
from pathlib import Path

2023-02-05 12:24:55.608961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-05 12:24:55.884725: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-05 12:25:00.296201: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib:/usr/

In [5]:
class DecoderFromNamedEntitySequence():
    def __init__(self, tokenizer, index_to_ner):
        self.tokenizer = tokenizer
        self.index_to_ner = index_to_ner

    def __call__(self, list_of_input_ids, list_of_pred_ids):
        input_token = self.tokenizer.decode_token_ids(list_of_input_ids)[0]
        pred_ner_tag = [self.index_to_ner[pred_id] for pred_id in list_of_pred_ids[0]]

        print("len: {}, input_token:{}".format(len(input_token), input_token))
        print("len: {}, pred_ner_tag:{}".format(len(pred_ner_tag), pred_ner_tag))

        # ----------------------------- parsing list_of_ner_word ----------------------------- #
        list_of_ner_word = []
        entity_word, entity_tag, prev_entity_tag = "", "", ""
        for i, pred_ner_tag_str in enumerate(pred_ner_tag):
            if "B-" in pred_ner_tag_str:
                entity_tag = pred_ner_tag_str[-3:]

                if prev_entity_tag != entity_tag and prev_entity_tag != "":
                    list_of_ner_word.append({"word": entity_word.replace("▁", " "), "tag": prev_entity_tag, "prob": None})

                entity_word = input_token[i]
                prev_entity_tag = entity_tag
            elif "I-"+entity_tag in pred_ner_tag_str:
                entity_word += input_token[i]
            else:
                if entity_word != "" and entity_tag != "":
                    list_of_ner_word.append({"word":entity_word.replace("▁", " "), "tag":entity_tag, "prob":None})
                entity_word, entity_tag, prev_entity_tag = "", "", ""


        # ----------------------------- parsing decoding_ner_sentence ----------------------------- #
        decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False

        for token_str, pred_ner_tag_str in zip(input_token, pred_ner_tag):
            token_str = token_str.replace('▁', ' ')  # '▁' 토큰을 띄어쓰기로 교체

            if 'B-' in pred_ner_tag_str:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'

                if token_str[0] == ' ':
                    token_str = list(token_str)
                    token_str[0] = ' <'
                    token_str = ''.join(token_str)
                    decoding_ner_sentence += token_str
                else:
                    decoding_ner_sentence += '<' + token_str
                is_prev_entity = True
                prev_entity_tag = pred_ner_tag_str[-3:] # 첫번째 예측을 기준으로 하겠음
                is_there_B_before_I = True

            elif 'I-' in pred_ner_tag_str:
                decoding_ner_sentence += token_str

                if is_there_B_before_I is True: # I가 나오기전에 B가 있어야하도록 체크
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token_str
                    is_prev_entity = False
                    is_there_B_before_I = False
                else:
                    decoding_ner_sentence += token_str

        return list_of_ner_word, decoding_ner_sentence

In [6]:
def ParseText(input_text):
    model_dir = Path("/home/work/nlp/keyword/model")
    model_config = Config(json_path=model_dir / 'config.json')

    tok_path = "/home/work/nlp/keyword/pytorch_bert_crf_ner/ptr_lm_model/tokenizer_78b3253a26.model"
    ptr_tokenizer = SentencepieceTokenizer(tok_path)

    # load vocab & tokenizer
    with open(model_dir / "vocab.pkl", 'rb') as f:
        vocab = pickle.load(f)

    tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=model_config.maxlen)

    # load ner_to_index.json
    with open(model_dir / "ner_to_index.json", 'rb') as f:
        ner_to_index = json.load(f)
        index_to_ner = {v: k for k, v in ner_to_index.items()}

    # Model
    # model = KobertSequenceFeatureExtractor(config=model_config, num_classes=len(ner_to_index))
    model = KobertCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)
    # model = KobertBiLSTMCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)
    # model = KobertBiGRUCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)

    # load
    model_dict = model.state_dict()
    # checkpoint = torch.load("./experiments/base_model/best-epoch-9-step-600-acc-0.845.bin", map_location=torch.device('cpu'))

    # checkpoint = torch.load("./experiments/base_model_with_crf/best-epoch-16-step-1500-acc-0.993.bin", map_location=torch.device('cpu'))
    checkpoint = torch.load("/home/work/nlp/keyword/model/best-epoch-12-step-1000-acc-0.960.bin", map_location=torch.device('cpu'))

    # checkpoint = torch.load("./experiments/base_model_with_bilstm_crf/best-epoch-15-step-2750-acc-0.992.bin", map_location=torch.device('cpu'))
    # checkpoint = torch.load("./experiments/base_model_with_bigru_crf/model-epoch-18-step-3250-acc-0.997.bin", map_location=torch.device('cpu'))

    convert_keys = {}

    for k, v in checkpoint['model_state_dict'].items():
        new_key_name = k.replace("module.", '')

        if new_key_name not in model_dict:
            print("{} is not int model_dict".format(new_key_name))
            continue
        convert_keys[new_key_name] = v

    model.load_state_dict(convert_keys)
    model.eval()
    device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    print(device)
    # n_gpu = torch.cuda.device_count()
    # if n_gpu > 1:
    #     model = torch.nn.DataParallel(model)

    decoder_from_res = DecoderFromNamedEntitySequence(tokenizer=tokenizer, index_to_ner=index_to_ner)
    
    
    list_of_input_ids = tokenizer.list_of_string_to_list_of_cls_sep_token_ids([input_text])
    x_input = torch.tensor(list_of_input_ids).long()

    ## for bert alone
    # y_pred = model(x_input)
    # list_of_pred_ids = y_pred.max(dim=-1)[1].tolist()

    ## for bert crf
    print(type(x_input))
    list_of_pred_ids = model(x_input)

    ## for bert bilstm crf & bert bigru crf
    # list_of_pred_ids = model(x_input, using_pack_sequence=False)

    list_of_ner_word, decoding_ner_sentence = decoder_from_res(list_of_input_ids=list_of_input_ids, list_of_pred_ids=list_of_pred_ids)
    print("list_of_ner_word:", list_of_ner_word)
    print("decoding_ner_sentence:", decoding_ner_sentence)
    
    return list_of_ner_word, decoding_ner_sentence

In [92]:
lst, dec = ParseText("미국 고용보고서 발표 이후 달러인덱스 113선 상승 반도체시장 악재 등 겹쳐 위험 회피 심리도 확대 아시아증시 하락에 코스피, 코스닥 2~4%대 내려 환율, 22.8원 급등 1430원대 상승…'킹달러'에 아통화 추락")

cuda:0
<class 'torch.Tensor'>
len: 62, input_token:['[CLS]', '▁미국', '▁고용', '보고서', '▁발표', '▁이후', '▁달러', '인', '덱', '스', '▁11', '3', '선', '▁상승', '▁반도체', '시장', '▁악', '재', '▁등', '▁', '겹', '쳐', '▁위험', '▁회', '피', '▁심리', '도', '▁확대', '▁아시아', '증', '시', '▁하락', '에', '▁코스피', ',', '▁코스닥', '▁2', '~', '4%', '대', '▁내려', '▁환율', ',', '▁22', '.', '8', '원', '▁급등', '▁14', '30', '원', '대', '▁상승', "...'", '킹', '달러', "'", '에', '▁아', '통화', '▁추락', '[SEP]']
len: 62, pred_ner_tag:['[CLS]', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-POH', 'O', 'B-POH', 'B-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MNY', 'I-MNY', 'I-MNY', 'O', 'O', 'B-MNY', 'I-MNY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
list_of_ner_word: [{'word': ' 미국', 'tag': 'ORG', 'prob': None}, {'word': ' 113', 'tag': 'NOH', 'prob': None}, {'word': ' 코스피', 'tag': 'POH', 'prob': None}, {'word': ' 코스닥', 'tag': 'P

In [13]:
print(lst)

[{'word': ' 코스피지수', 'tag': 'POH', 'prob': None}, {'word': ' 3거래일', 'tag': 'DUR', 'prob': None}, {'word': ' 1%', 'tag': 'PNT', 'prob': None}, {'word': ' 26일', 'tag': 'DAT', 'prob': None}, {'word': ' 코스피', 'tag': 'POH', 'prob': None}, {'word': ' 1%', 'tag': 'PNT', 'prob': None}, {'word': ' 12월 1일', 'tag': 'DAT', 'prob': None}, {'word': ' 2460', 'tag': 'MNY', 'prob': None}]


In [6]:
import json

In [13]:
with open('/home/work/nlp/keyword/news_keyword.json', 'r') as f:
    json_data = json.load(f)

print(json_data['2018-01-01'] )

[['외국인코스피', '전망', '시장 전문가들은 연초 국내 증시가 연말 단기 이슈와 이벤트 장세에서 펀더멘털 기초여건 장세로 전환할 것으로 진단했다.증시 1월 효과 있을까 코스피 2450∼2500선 등락 전망'], ['외국인코스피', '전망', '시장 전문가들은 연초 국내 증시가 연말 단기 이슈와 이벤트 장세에서 펀더멘털 기초여건 장세로 전환할 것으로 진단했다.증시 1월 효과 있을까 코스피 2450∼2500선 등락 전망'], ['국내증시', '기대감', '서울 연합뉴스 유현민 기자 새해 첫 주 2∼5일 국내 증시는 1월 효과와 실적 기대감에 대체로 호조세를 이어갈 전망이다. 새해 첫주 증시 1월 효과 실적 기대에 호조세 전망']]


In [11]:
with open('2006-01-01_2022-10-31_sum.json', 'r') as f:
    json_data = json.load(f)

print(json_data['2015-01-02'] )

['2일 동부그룹 비금융 계열 지주회사 격인 동부CNI는 전 거래일(지난해 12월 30일)에 비해 11.75% 내린 2215원에 마감했다. 동부라이텍 감소11.76%(52주 신저가), 동부로봇 감소8.02%, 동부제철 감소0.71%, 동부증권 감소0.54%, 동부화재 감소0.18% 등 대부분 동부그룹주가 하락했다. 동부하이텍만 유일하게 0.65% 상승했다. 법정관리를 신청한 동부건설 거래는 정지됐다. 동부그룹주는 유가증권시장과 코스닥시장에 각각 6개사와 2개사가 상장돼 있다.동부그룹주 일제히 하락', '주가연계증권(ELS)이 연간 발행액 70조원이라는 새 역사를 쓰면서 국내 간접 금융투자 시장의 ‘판’을 바꾸고 있다. ELS는 박스권 증시에서도 연 6~8% 수준으로 안정적인 수익 추구가 가능하기 때문이다. 올해 증시도 횡보장세가 예상되면서 ELS 발행 규모가 85조원까지 늘어날 것이란 전망이 나온다.ELS 70조 시대…‘간접투자 대명사’로', "새해 국내외 투자 환경 점검 핵심테마 '지배구조 변화감소배당 확대'", '조선주, 1년 장사 시작도 전에 \'울상\'…"대우조선 빼고는..."', "2015년 증시 키워드 '고령화·인플레이션'", "청양의 해에도 '형'보다 '아우' 선전할 것", "지난해말 '지켜보기'로 일관하던 투자자들이 새해 주식시장에서는 어떤 행보를 보일지 관심이 쏠리고 있다.새해 첫 쇼핑…장바구니 담을 종목은", '전문가들은 “실적 등 밸류에이션 개선 가능성이 높은 업종과 종목 위주로 새해 투자 포트폴리오를 구성하는 것이 바람직하다”고 조언한다.2014년이어 2015년 실적 개선 꾸준한 종목 ‘주목’', '2015 첫 증시, 1시간 늦은 10시 개장…시간외시장 정상운영', '2일 코스피는 전 거래일 대비 10.84포인트(0.57%) 오른 1926.44에 거래를 마쳤다. 지수는 종가기준 2거래일만에 1920선 위로 다시 올라섰다.코스피, 새해 첫날 상승 마감…삼성그룹주 강세', '2015년 을미년 첫 거래일에서 코스피·코스닥지수가 동반상승하는 순조

In [12]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
kw_model = KeyBERT(model=sentence_model)

In [8]:
from konlpy.tag import Hannanum
from konlpy.tag import Mecab
from bs4 import BeautifulSoup
from nltk import collocations
from konlpy.utils import pprint
import requests
import time
import hanja
import re
from datetime import datetime, timedelta

mecab = Mecab()
hannanum = Hannanum()

In [9]:
def clean_text(new):
    new = new.replace('美', '미국')
    new = new.replace('韓', '한국')
    new = new.replace('中', '중국')
    new = new.replace('日', '일본')
    new = new.replace('英', '영국')
    new = new.replace('獨', '독일')
    new = new.replace('强', '강세')
    new = new.replace('↑', ' 상승')
    new = new.replace('↓', ' 하락')
    new = new.replace('-', '감소')
    new = new.replace('&amp;', '&')
    new = re.sub('\[.*\]', '', new)
    new = re.sub("[-=+,#/\?:^@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·◆]▶ⓒ◆"," ",new)
    new = ' '.join(new.split())
    new = hanja.translate(new, 'substitution')
    return new

In [ ]:
# crawling News

def crawling_news(start, last):

    start_date = datetime.strptime(start, '%Y-%m-%d')
    last_date = datetime.strptime(last, '%Y-%m-%d')

    data = {}
    split_data = {}
    new_string_split = []

    while start_date <= last_date:
        dates = start_date.strftime('%Y-%m-%d')
        print(dates)
        
        if json_data.get(dates) != None:
            data[dates] = json_data[dates]
            start_date += timedelta(days=1)
            continue
            
        
        url = f"https://finance.naver.com/news/mainnews.naver"
        req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}, params={'date': dates})
        html = BeautifulSoup(req.text, 'lxml')
        news = html.find_all('dd', class_="articleSubject")
        news += html.find_all('dt', class_="articleSubject")
        news_title=[]
        news_title_split=[]
        for new in news:
            new = new.find('a')
            news_href = "https://finance.naver.com" + new.attrs['href']
            new_string = ""

            # subtracting news summary or first paragraph
            req = requests.get(news_href, headers={'User-agent': 'Mozilla/5.0'})
            html = BeautifulSoup(req.text, 'lxml')
            summary = html.find(class_="media_end_summary")

            '''
            article = html.find('div', id="content")
            print(article)
            print(article.get_text())
            new_string = clean_text(article.get_text())
            new_string_split = str(new_string).split('.')
            '''

            if summary:
                summary = re.sub('<[^>]+>'," ",str(summary))
                new_string = summary
                new_string_split = summary.split('.')
            else:
                article = html.find(class_="articleCont")
                if article is not None and article.find('span') is not None:
                    article.find('span').decompose()
                    article = re.sub('<div class="articleCont" id="content">',"",str(article))
                    article = re.search('<[^>]+>[^<]+<[^>]+>', str(article))
                    article = re.sub('<[^>]+>',"",str(article.group()))
                    article = re.sub('<[^>]+>'," ",str(article))
                    new_string = article
                    new_string_split = str(article).split('.')
          

            # subtracting news title
            new_string += new.string
            new_string_split.append(new.string)
            new_string = clean_text(new_string)

            for sent in new_string_split:
                news_title_split.append(clean_text(sent))
            news_title.append(new_string)
    
        data[dates] = news_title
        split_data[dates] = news_title_split
    
        time.sleep(0.005)
        start_date += timedelta(days=1)

    return data, split_data

START_DATE = '2006-01-01'
END_DATE = '2022-10-31'
data, split_data = crawling_news(START_DATE, END_DATE)

with open(START_DATE + '_' + END_DATE + "_sum.json", 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False)

2006-01-01
2006-01-02
2006-01-03
2006-01-04
2006-01-05
2006-01-06
2006-01-07
2006-01-08
2006-01-09
2006-01-10
2006-01-11
2006-01-12
2006-01-13
2006-01-14
2006-01-15
2006-01-16
2006-01-17
2006-01-18
2006-01-19
2006-01-20
2006-01-21
2006-01-22
2006-01-23
2006-01-24
2006-01-25
2006-01-26
2006-01-27
2006-01-28
2006-01-29
2006-01-30
2006-01-31
2006-02-01
2006-02-02
2006-02-03
2006-02-04
2006-02-05
2006-02-06
2006-02-07
2006-02-08
2006-02-09
2006-02-10
2006-02-11
2006-02-12
2006-02-13
2006-02-14
2006-02-15
2006-02-16
2006-02-17
2006-02-18
2006-02-19
2006-02-20
2006-02-21
2006-02-22
2006-02-23
2006-02-24
2006-02-25
2006-02-26
2006-02-27
2006-02-28
2006-03-01
2006-03-02
2006-03-03
2006-03-04
2006-03-05
2006-03-06
2006-03-07
2006-03-08
2006-03-09
2006-03-10
2006-03-11
2006-03-12
2006-03-13
2006-03-14
2006-03-15
2006-03-16
2006-03-17
2006-03-18
2006-03-19
2006-03-20
2006-03-21
2006-03-22
2006-03-23
2006-03-24
2006-03-25
2006-03-26
2006-03-27
2006-03-28
2006-03-29
2006-03-30
2006-03-31
2006-04-01

2008-01-16
2008-01-17
2008-01-18
2008-01-19
2008-01-20
2008-01-21
2008-01-22
2008-01-23
2008-01-24
2008-01-25
2008-01-26
2008-01-27
2008-01-28
2008-01-29
2008-01-30
2008-01-31
2008-02-01
2008-02-02
2008-02-03
2008-02-04
2008-02-05
2008-02-06
2008-02-07
2008-02-08
2008-02-09
2008-02-10
2008-02-11
2008-02-12
2008-02-13
2008-02-14
2008-02-15
2008-02-16
2008-02-17
2008-02-18
2008-02-19
2008-02-20
2008-02-21
2008-02-22
2008-02-23
2008-02-24
2008-02-25
2008-02-26
2008-02-27
2008-02-28
2008-02-29
2008-03-01
2008-03-02
2008-03-03
2008-03-04
2008-03-05
2008-03-06
2008-03-07
2008-03-08
2008-03-09
2008-03-10
2008-03-11
2008-03-12
2008-03-13
2008-03-14
2008-03-15
2008-03-16
2008-03-17
2008-03-18
2008-03-19
2008-03-20
2008-03-21
2008-03-22
2008-03-23
2008-03-24
2008-03-25
2008-03-26
2008-03-27
2008-03-28
2008-03-29
2008-03-30
2008-03-31
2008-04-01
2008-04-02
2008-04-03
2008-04-04
2008-04-05
2008-04-06
2008-04-07
2008-04-08
2008-04-09
2008-04-10
2008-04-11
2008-04-12
2008-04-13
2008-04-14
2008-04-15

2010-01-30
2010-01-31
2010-02-01
2010-02-02
2010-02-03
2010-02-04
2010-02-05
2010-02-06
2010-02-07
2010-02-08
2010-02-09
2010-02-10
2010-02-11
2010-02-12
2010-02-13
2010-02-14
2010-02-15
2010-02-16
2010-02-17
2010-02-18
2010-02-19
2010-02-20
2010-02-21
2010-02-22
2010-02-23
2010-02-24
2010-02-25
2010-02-26
2010-02-27
2010-02-28
2010-03-01
2010-03-02
2010-03-03
2010-03-04
2010-03-05
2010-03-06
2010-03-07
2010-03-08
2010-03-09
2010-03-10
2010-03-11
2010-03-12
2010-03-13
2010-03-14
2010-03-15
2010-03-16
2010-03-17
2010-03-18
2010-03-19
2010-03-20
2010-03-21
2010-03-22
2010-03-23
2010-03-24
2010-03-25
2010-03-26
2010-03-27
2010-03-28
2010-03-29
2010-03-30
2010-03-31
2010-04-01
2010-04-02
2010-04-03
2010-04-04
2010-04-05
2010-04-06
2010-04-07
2010-04-08
2010-04-09
2010-04-10
2010-04-11
2010-04-12
2010-04-13
2010-04-14
2010-04-15
2010-04-16
2010-04-17
2010-04-18
2010-04-19
2010-04-20
2010-04-21
2010-04-22
2010-04-23
2010-04-24
2010-04-25
2010-04-26
2010-04-27
2010-04-28
2010-04-29
2010-04-30

2012-02-14
2012-02-15
2012-02-16
2012-02-17
2012-02-18
2012-02-19
2012-02-20
2012-02-21
2012-02-22
2012-02-23
2012-02-24
2012-02-25
2012-02-26
2012-02-27
2012-02-28
2012-02-29
2012-03-01
2012-03-02
2012-03-03
2012-03-04
2012-03-05
2012-03-06
2012-03-07
2012-03-08
2012-03-09
2012-03-10
2012-03-11
2012-03-12
2012-03-13
2012-03-14
2012-03-15
2012-03-16
2012-03-17
2012-03-18
2012-03-19
2012-03-20
2012-03-21
2012-03-22
2012-03-23
2012-03-24
2012-03-25
2012-03-26
2012-03-27
2012-03-28
2012-03-29
2012-03-30
2012-03-31
2012-04-01
2012-04-02
2012-04-03
2012-04-04
2012-04-05
2012-04-06
2012-04-07
2012-04-08
2012-04-09
2012-04-10
2012-04-11
2012-04-12
2012-04-13
2012-04-14
2012-04-15
2012-04-16
2012-04-17
2012-04-18
2012-04-19
2012-04-20
2012-04-21
2012-04-22
2012-04-23
2012-04-24
2012-04-25
2012-04-26
2012-04-27
2012-04-28
2012-04-29
2012-04-30
2012-05-01
2012-05-02
2012-05-03
2012-05-04
2012-05-05
2012-05-06
2012-05-07
2012-05-08
2012-05-09
2012-05-10
2012-05-11
2012-05-12
2012-05-13
2012-05-14

In [13]:
keyword_pair = {}
stopword = ['증시', '지수', '코스피', '코스닥', '급락', '하락세']

In [14]:
# extracting news keywords

start = '2006-01-01'
last = '2022-10-31'

start_date = datetime.strptime(start, '%Y-%m-%d')
last_date = datetime.strptime(last, '%Y-%m-%d')

while start_date <= last_date:
    dates = start_date.strftime('%Y-%m-%d')
    
    if dates in keyword_pair:
        start_date += timedelta(days=1)
        continue

    # extracting keyword 
    
    news_list = data[dates]
    keywords_count = {}
    for news in news_list:
        news = " ".join(mecab.morphs(news))
        #print(mecab.morphs(news))
        keywords = kw_model.extract_keywords(news, top_n=10, stop_words = stopword)
        #print(keywords)

        for keyword in keywords:
            hannanum_list = hannanum.analyze(keyword[0])
            NCP = False
            
            if keyword[0].isdigit():
                NCP = True

            for temp1 in hannanum_list:
                for temp2 in temp1:
                    for analyze_word in temp2:
                        if analyze_word[1] == 'ncpa' or analyze_word[1] == 'ncps':
                            NCP = True           
            if NCP == False :
                if keyword[0] in keywords_count:
                    keywords_count[keyword[0]][0] += 1
                    if keywords_count[keyword[0]][1] < keyword[1] : 
                        keywords_count[keyword[0]][1] = keyword[1]
                else:
                    keywords_count[keyword[0]] = [1, keyword[1]]
    
    keywords_count = sorted(keywords_count.items(), key = lambda item:( (-item[1][0]), (-item[1][1])))
    i = 0
    print(dates)
    #print(keywords_count)
    #print()
    if(len(keywords_count)==0):
        keyword_pair[dates] = ["", ""]
        start_date += timedelta(days=1)
        continue

    top_keyword = keywords_count[0][0]
    if len(keywords_count) > 2:
        second_keyword = keywords_count[1][0]
        third_keyword = keywords_count[2][0]
    
    # select news title using term-frequency
    
    top_title = []
    sec_title = []
    third_title = []
    
    for news in news_list:
        if top_keyword in news:
            top_title.append(news)
            if second_keyword in news:
                sec_title.append(news)
                if third_keyword in news:
                    third_title.append(news)
    
    keyword = ""
    if len(third_title) > 0 :
        keyword_pair[dates] = third_title[0]
    elif len(sec_title) > 0 :
        keyword_pair[dates] = sec_title[0]
    elif len(top_title) > 0 :
        keyword_pair[dates] = top_title[0]
    
    
    start_date += timedelta(days=1)
    
with open(start + '_' + last + "_keyword.json", 'w') as outfile:
    json.dump(keyword_pair, outfile, ensure_ascii=False)

2006-01-01
2006-01-02
2006-01-03
2006-01-04
2006-01-05
2006-01-06
2006-01-07
2006-01-08
2006-01-09
2006-01-10
2006-01-11
2006-01-12
2006-01-13
2006-01-14
2006-01-15
2006-01-16
2006-01-17
2006-01-18
2006-01-19
2006-01-20
2006-01-21
2006-01-22
2006-01-23
2006-01-24
2006-01-25
2006-01-26
2006-01-27
2006-01-28
2006-01-29
2006-01-30
2006-01-31
2006-02-01
2006-02-02
2006-02-03
2006-02-04
2006-02-05
2006-02-06
2006-02-07
2006-02-08
2006-02-09
2006-02-10
2006-02-11
2006-02-12
2006-02-13
2006-02-14
2006-02-15
2006-02-16
2006-02-17
2006-02-18
2006-02-19
2006-02-20
2006-02-21
2006-02-22
2006-02-23
2006-02-24
2006-02-25
2006-02-26
2006-02-27
2006-02-28
2006-03-01
2006-03-02
2006-03-03
2006-03-04
2006-03-05
2006-03-06
2006-03-07
2006-03-08
2006-03-09
2006-03-10
2006-03-11
2006-03-12
2006-03-13
2006-03-14
2006-03-15
2006-03-16
2006-03-17
2006-03-18
2006-03-19
2006-03-20
2006-03-21
2006-03-22
2006-03-23
2006-03-24
2006-03-25
2006-03-26
2006-03-27
2006-03-28
2006-03-29
2006-03-30
2006-03-31
2006-04-01

2008-01-16
2008-01-17
2008-01-18
2008-01-19
2008-01-20
2008-01-21
2008-01-22
2008-01-23
2008-01-24
2008-01-25
2008-01-26
2008-01-27
2008-01-28
2008-01-29
2008-01-30
2008-01-31
2008-02-01
2008-02-02
2008-02-03
2008-02-04
2008-02-05
2008-02-06
2008-02-07
2008-02-08
2008-02-09
2008-02-10
2008-02-11
2008-02-12
2008-02-13
2008-02-14
2008-02-15
2008-02-16
2008-02-17
2008-02-18
2008-02-19
2008-02-20
2008-02-21
2008-02-22
2008-02-23
2008-02-24
2008-02-25
2008-02-26
2008-02-27
2008-02-28
2008-02-29
2008-03-01
2008-03-02
2008-03-03
2008-03-04
2008-03-05
2008-03-06
2008-03-07
2008-03-08
2008-03-09
2008-03-10
2008-03-11
2008-03-12
2008-03-13
2008-03-14
2008-03-15
2008-03-16
2008-03-17
2008-03-18
2008-03-19
2008-03-20
2008-03-21
2008-03-22
2008-03-23
2008-03-24
2008-03-25
2008-03-26
2008-03-27
2008-03-28
2008-03-29
2008-03-30
2008-03-31
2008-04-01
2008-04-02
2008-04-03
2008-04-04
2008-04-05
2008-04-06
2008-04-07
2008-04-08
2008-04-09
2008-04-10
2008-04-11
2008-04-12
2008-04-13
2008-04-14
2008-04-15

2010-01-30
2010-01-31
2010-02-01
2010-02-02
2010-02-03
2010-02-04
2010-02-05
2010-02-06
2010-02-07
2010-02-08
2010-02-09
2010-02-10
2010-02-11
2010-02-12
2010-02-13
2010-02-14
2010-02-15
2010-02-16
2010-02-17
2010-02-18
2010-02-19
2010-02-20
2010-02-21
2010-02-22
2010-02-23
2010-02-24
2010-02-25
2010-02-26
2010-02-27
2010-02-28
2010-03-01
2010-03-02
2010-03-03
2010-03-04
2010-03-05
2010-03-06
2010-03-07
2010-03-08
2010-03-09
2010-03-10
2010-03-11
2010-03-12
2010-03-13
2010-03-14
2010-03-15
2010-03-16
2010-03-17
2010-03-18
2010-03-19
2010-03-20
2010-03-21
2010-03-22
2010-03-23
2010-03-24
2010-03-25
2010-03-26
2010-03-27
2010-03-28
2010-03-29
2010-03-30
2010-03-31
2010-04-01
2010-04-02
2010-04-03
2010-04-04
2010-04-05
2010-04-06
2010-04-07
2010-04-08
2010-04-09
2010-04-10
2010-04-11
2010-04-12
2010-04-13
2010-04-14
2010-04-15
2010-04-16
2010-04-17
2010-04-18
2010-04-19
2010-04-20
2010-04-21
2010-04-22
2010-04-23
2010-04-24
2010-04-25
2010-04-26
2010-04-27
2010-04-28
2010-04-29
2010-04-30

2012-02-14
2012-02-15
2012-02-16
2012-02-17
2012-02-18
2012-02-19
2012-02-20
2012-02-21
2012-02-22
2012-02-23
2012-02-24
2012-02-25
2012-02-26
2012-02-27
2012-02-28
2012-02-29
2012-03-01
2012-03-02
2012-03-03
2012-03-04
2012-03-05
2012-03-06
2012-03-07
2012-03-08
2012-03-09
2012-03-10
2012-03-11
2012-03-12
2012-03-13
2012-03-14
2012-03-15
2012-03-16
2012-03-17
2012-03-18
2012-03-19
2012-03-20
2012-03-21
2012-03-22
2012-03-23
2012-03-24
2012-03-25
2012-03-26
2012-03-27
2012-03-28
2012-03-29
2012-03-30
2012-03-31
2012-04-01
2012-04-02
2012-04-03
2012-04-04
2012-04-05
2012-04-06
2012-04-07
2012-04-08
2012-04-09
2012-04-10
2012-04-11
2012-04-12
2012-04-13
2012-04-14
2012-04-15
2012-04-16
2012-04-17
2012-04-18
2012-04-19
2012-04-20
2012-04-21
2012-04-22
2012-04-23
2012-04-24
2012-04-25
2012-04-26
2012-04-27
2012-04-28
2012-04-29
2012-04-30
2012-05-01
2012-05-02
2012-05-03
2012-05-04
2012-05-05
2012-05-06
2012-05-07
2012-05-08
2012-05-09
2012-05-10
2012-05-11
2012-05-12
2012-05-13
2012-05-14

2014-02-28
2014-03-01
2014-03-02
2014-03-03
2014-03-04
2014-03-05
2014-03-06
2014-03-07
2014-03-08
2014-03-09
2014-03-10
2014-03-11
2014-03-12
2014-03-13
2014-03-14
2014-03-15
2014-03-16
2014-03-17
2014-03-18
2014-03-19
2014-03-20
2014-03-21
2014-03-22
2014-03-23
2014-03-24
2014-03-25
2014-03-26
2014-03-27
2014-03-28
2014-03-29
2014-03-30
2014-03-31
2014-04-01
2014-04-02
2014-04-03
2014-04-04
2014-04-05
2014-04-06
2014-04-07
2014-04-08
2014-04-09
2014-04-10
2014-04-11
2014-04-12
2014-04-13
2014-04-14
2014-04-15
2014-04-16
2014-04-17
2014-04-18
2014-04-19
2014-04-20
2014-04-21
2014-04-22
2014-04-23
2014-04-24
2014-04-25
2014-04-26
2014-04-27
2014-04-28
2014-04-29
2014-04-30
2014-05-01
2014-05-02
2014-05-03
2014-05-04
2014-05-05
2014-05-06
2014-05-07
2014-05-08
2014-05-09
2014-05-10
2014-05-11
2014-05-12
2014-05-13
2014-05-14
2014-05-15
2014-05-16
2014-05-17
2014-05-18
2014-05-19
2014-05-20
2014-05-21
2014-05-22
2014-05-23
2014-05-24
2014-05-25
2014-05-26
2014-05-27
2014-05-28
2014-05-29

2016-03-14
2016-03-15
2016-03-16
2016-03-17
2016-03-18
2016-03-19
2016-03-20
2016-03-21
2016-03-22
2016-03-23
2016-03-24
2016-03-25
2016-03-26
2016-03-27
2016-03-28
2016-03-29
2016-03-30
2016-03-31
2016-04-01
2016-04-02
2016-04-03
2016-04-04
2016-04-05
2016-04-06
2016-04-07
2016-04-08
2016-04-09
2016-04-10
2016-04-11
2016-04-12
2016-04-13
2016-04-14
2016-04-15
2016-04-16
2016-04-17
2016-04-18
2016-04-19
2016-04-20
2016-04-21
2016-04-22
2016-04-23
2016-04-24
2016-04-25
2016-04-26
2016-04-27
2016-04-28
2016-04-29
2016-04-30
2016-05-01
2016-05-02
2016-05-03
2016-05-04
2016-05-05
2016-05-06
2016-05-07
2016-05-08
2016-05-09
2016-05-10
2016-05-11
2016-05-12
2016-05-13
2016-05-14
2016-05-15
2016-05-16
2016-05-17
2016-05-18
2016-05-19
2016-05-20
2016-05-21
2016-05-22
2016-05-23
2016-05-24
2016-05-25
2016-05-26
2016-05-27
2016-05-28
2016-05-29
2016-05-30
2016-05-31
2016-06-01
2016-06-02
2016-06-03
2016-06-04
2016-06-05
2016-06-06
2016-06-07
2016-06-08
2016-06-09
2016-06-10
2016-06-11
2016-06-12

2018-03-29
2018-03-30
2018-03-31
2018-04-01
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-07
2018-04-08
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-14
2018-04-15
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-21
2018-04-22
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-28
2018-04-29
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-05
2018-05-06
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-12
2018-05-13
2018-05-14
2018-05-15
2018-05-16
2018-05-17
2018-05-18
2018-05-19
2018-05-20
2018-05-21
2018-05-22
2018-05-23
2018-05-24
2018-05-25
2018-05-26
2018-05-27
2018-05-28
2018-05-29
2018-05-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27

2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-13
2020-06-14
2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
2020-06-20
2020-06-21
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12

2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-11
2022-06-12
2022-06-13
2022-06-14
2022-06-15
2022-06-16
2022-06-17
2022-06-18
2022-06-19
2022-06-20
2022-06-21
2022-06-22
2022-06-23
2022-06-24
2022-06-25
2022-06-26
2022-06-27
2022-06-28
2022-06-29
2022-06-30
2022-07-01
2022-07-02
2022-07-03
2022-07-04
2022-07-05
2022-07-06
2022-07-07
2022-07-08
2022-07-09
2022-07-10
2022-07-11
2022-07-12
2022-07-13
2022-07-14
2022-07-15
2022-07-16
2022-07-17
2022-07-18
2022-07-19
2022-07-20
2022-07-21
2022-07-22
2022-07-23
2022-07-24
2022-07-25
2022-07-26
2022-07-27

In [74]:
keyword_pair

{'2022-10-10': '다우는 0.14% 상승해 혼조세로 시작 AMD 등 주요 반도체주 하락세 지속 대형 은행 등 어닝 발표로 시즌 개막 CPI·어닝 앞두고 반도체 약세…나스닥 감소0.60% 출발',
 '2022-10-11': "미국 고용보고서 발표 이후 달러인덱스 113선 상승 반도체시장 악재 등 겹쳐 위험 회피 심리도 확대 아시아증시 하락에 코스피, 코스닥 2~4%대 내려 환율, 22.8원 급등 1430원대 상승…'킹달러'에 아통화 추락",
 '2022-10-12': 'BOE 개별 은행에 채권 매입 연장 가능성 소식 달러인덱스 113선에서 하락 전환 강달러 제한 이창용 총재 "1400원대 해외투자 상투 가능성" 강달러 되돌림+이 총재 고환율 경계 발언…환율, 1420원대 급락',
 '2022-10-13': '카카오 4형제 신저가…"긴축·경기침체 이중고" 달러·원 환율 1431원으로 올라 코스피, 미국 물가 경계에 1.8% 하락…코스닥, 2년5개월만에 최저',
 '2022-10-14': "14일 국내 증시는 미국 증시의 반등에 기대어 상승 출발이 예상되지만, 장중 변동성을 보일 것으로 전망된다. 원·달러 환율 등의 변수에 따라 방향성은 결정될 것으로 보인다. 여전히 높은 수준의 물가 지표가 부담으로 작용하고 있는 가운데 경기 침체도 변동성을 자극할 전망이다.'최악 인플레' 견딘 미국 반등에 기댄다…되돌림에도 변동성 불가피",
 '2022-10-15': '뉴욕증시가 역사적인 반등 랠리를 기록한 지 하루 만에 다시 큰 폭으로 하락했다. 10년물 미 국채금리는 다시 4%를 돌파했다. "인플레 우려" 반등랠리 하루만에 급락...테슬라 7% 하락',
 '2022-10-16': '지난주 월가는 JP모건체이스, 웰스 파고, 씨티그룹 등 주요 금융기업들의 예상보다 좋은 실적을 확인하며 어닝 시즌을 시작했다. 이번주에는 골드만삭스, 뱅크오브아메리카, 프록터 앤 갬블, 테슬라, 넷플릭스, 유나이티드항공, 존슨 앤 존슨 등이 실적을 내놓는다. "험악한 분위기, 증시 바닥 